# Elastic Net

In [12]:
#Omid55
import requests
import pandas as pd
import pickle as pk
import csv
import time
import numpy as np
import random
from collections import defaultdict
import seaborn as sns
import matplotlib.pylab as plt
from scipy.sparse import csr_matrix
import networkx as nx
import glob
import networkx
import scipy
import classification_pipeline as pi
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
% matplotlib inline

In [42]:
file_names = ['DATA_2016_Nodes.csv', 'DATA_2016_Networks.csv', 'DATA_2016_Nodes+Networks.csv']

In [113]:
net_metric_names = ['average_weight', 'std_weight', 'AC', 'clustering', 'shortest_path_length', 'diameter', 'spectrum_v1', 'spectrum_v2', 'spectrum_v3', 'spectrum_v4', 'spectrum_v5']

node_names = ['avg_game_count', 'std_game_count', 'avg_win_ratio', 'std_win_ratio', 'avg_same_champion_game_count', 
             'std_same_champion_game_count', 'avg_same_champion_win_ratio', 'std_same_champion_win_ratio']

names = node_names + net_metric_names


dataset0_feature_names = ['winner_' + na for na in node_names] + ['loser_' + na for na in node_names]
dataset1_feature_names = ['winner_' + na for na in net_metric_names] + ['loser_' + na for na in net_metric_names]
dataset2_feature_names = ['winner_' + na for na in names] + ['loser_' + na for na in names]

assert len(dataset0_feature_names) == dataset0.shape[1]-1, "ERROR0"
assert len(dataset1_feature_names) == dataset1.shape[1]-1, "ERROR1"
assert len(dataset2_feature_names) == dataset2.shape[1]-1, "ERROR2"

In [68]:
dataset0 = pi.init(file_names[0])
dataset1 = pi.init(file_names[1])
dataset2 = pi.init(file_names[2])

Starting ...
Loaded data size:  (577, 17)
Swapping  8.0 : 16.0  with 0 : 8.0  ...
Dataset size:  (577, 17)
Starting ...
Loaded data size:  (577, 23)
Swapping  11.0 : 22.0  with 0 : 11.0  ...
Dataset size:  (577, 23)
Starting ...
Loaded data size:  (577, 39)
Swapping  19.0 : 38.0  with 0 : 19.0  ...
Dataset size:  (577, 39)


In [130]:
def apply_enet(dt, dt_feature_names, alpha=0.75, l1_ratio=0.7):
    enet = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    train_size = 0.63*len(dataset0)
    # TODO: MAKE IT 10-FOLD AND ADD VALUES OF ACCURACIES ...
    X_train = dt.ix[:train_size,:-1]
    y_train = dt.ix[:train_size,-1]
    X_test = dt.ix[train_size:,:-1]
    y_test = dt.ix[train_size:,-1]
    
    y_pred_enet = enet.fit(X_train, y_train).predict(X_test)
    r2_score_enet = r2_score(y_test, y_pred_enet)
    print(enet)
    print("r^2 on test data : %f\n" % r2_score_enet)
    
    selected_list = [x for (y,x) in sorted(zip(abs(enet.coef_), dt.columns), reverse=True) if y!=0]
    print([dt_feature_names[int(i)] for i in selected_list])

In [131]:
print 'ALL features:', dataset0_feature_names, '\n'
apply_enet(dataset0, dataset0_feature_names, alpha=1, l1_ratio=0.045)

ALL features: ['winner_avg_game_count', 'winner_std_game_count', 'winner_avg_win_ratio', 'winner_std_win_ratio', 'winner_avg_same_champion_game_count', 'winner_std_same_champion_game_count', 'winner_avg_same_champion_win_ratio', 'winner_std_same_champion_win_ratio', 'loser_avg_game_count', 'loser_std_game_count', 'loser_avg_win_ratio', 'loser_std_win_ratio', 'loser_avg_same_champion_game_count', 'loser_std_same_champion_game_count', 'loser_avg_same_champion_win_ratio', 'loser_std_same_champion_win_ratio'] 

ElasticNet(alpha=1, copy_X=True, fit_intercept=True, l1_ratio=0.045,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
r^2 on test data : 0.003323

['loser_avg_same_champion_game_count', 'winner_std_same_champion_win_ratio', 'loser_avg_game_count', 'loser_avg_same_champion_win_ratio']


In [133]:
print 'ALL features:', dataset1_feature_names, '\n'
apply_enet(dataset1, dataset1_feature_names, alpha=1, l1_ratio=0.05)

ALL features: ['winner_average_weight', 'winner_std_weight', 'winner_AC', 'winner_clustering', 'winner_shortest_path_length', 'winner_diameter', 'winner_spectrum_v1', 'winner_spectrum_v2', 'winner_spectrum_v3', 'winner_spectrum_v4', 'winner_spectrum_v5', 'loser_average_weight', 'loser_std_weight', 'loser_AC', 'loser_clustering', 'loser_shortest_path_length', 'loser_diameter', 'loser_spectrum_v1', 'loser_spectrum_v2', 'loser_spectrum_v3', 'loser_spectrum_v4', 'loser_spectrum_v5'] 

ElasticNet(alpha=1, copy_X=True, fit_intercept=True, l1_ratio=0.05,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
r^2 on test data : -0.005825

['loser_std_weight', 'loser_spectrum_v5', 'loser_average_weight']


In [136]:
print 'ALL features:', dataset2_feature_names, '\n'
apply_enet(dataset2, dataset2_feature_names, alpha=1, l1_ratio=0.04)

ALL features: ['winner_avg_game_count', 'winner_std_game_count', 'winner_avg_win_ratio', 'winner_std_win_ratio', 'winner_avg_same_champion_game_count', 'winner_std_same_champion_game_count', 'winner_avg_same_champion_win_ratio', 'winner_std_same_champion_win_ratio', 'winner_average_weight', 'winner_std_weight', 'winner_AC', 'winner_clustering', 'winner_shortest_path_length', 'winner_diameter', 'winner_spectrum_v1', 'winner_spectrum_v2', 'winner_spectrum_v3', 'winner_spectrum_v4', 'winner_spectrum_v5', 'loser_avg_game_count', 'loser_std_game_count', 'loser_avg_win_ratio', 'loser_std_win_ratio', 'loser_avg_same_champion_game_count', 'loser_std_same_champion_game_count', 'loser_avg_same_champion_win_ratio', 'loser_std_same_champion_win_ratio', 'loser_average_weight', 'loser_std_weight', 'loser_AC', 'loser_clustering', 'loser_shortest_path_length', 'loser_diameter', 'loser_spectrum_v1', 'loser_spectrum_v2', 'loser_spectrum_v3', 'loser_spectrum_v4', 'loser_spectrum_v5'] 

ElasticNet(alpha=1